# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

In [124]:
import pandas as pd
import xml.etree.ElementTree as ET
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag

## Задачи для совместного разбора

1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.

In [10]:
obj = {
    "home_page": "https://github.com/pypa/sampleproject",
    "keywords": "sample setuptools development",
    "license": "MIT",
}
print(f"{list(obj.keys())[0]} = {obj['home_page']}\n{list(obj.keys())[1]} = {obj['keywords']}\n{list(obj.keys())[2]} = {obj['license']}")

home_page = https://github.com/pypa/sampleproject
keywords = sample setuptools development
license = MIT


2. Написать регулярное выражение,которое позволит найти номера групп студентов.

In [4]:
obj = pd.Series(["Евгения гр.ПМ19-1", "Илья пм 20-4", "Анна 20-3"])
obj

0    Евгения гр.ПМ19-1
1         Илья пм 20-4
2            Анна 20-3
dtype: object

3. Разбейте текст формулировки задачи 2 на слова.

## Лабораторная работа 6

### Форматирование строк

1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [113]:
recipes = pd.read_csv('recipes_sample.csv')
recipes_id_min = recipes.iloc[:, [1, 2]]
recipes_sample = recipes_id_min.sample(5)
max_lenght = 0
for id_ in recipes_sample.iloc[:, 0]:
    max_lenght = max(max_lenght, len(str(id_)))
print(f"|{'id' :^{max_lenght + 4}}| {'minutes' :^{max_lenght + 4}}|\n|{'':-^{(2 * max_lenght) + 10}}|")
for i in range(5):
    print(f"|{list(recipes_sample.iloc[:, 0])[i]:^{max_lenght + 4}}| {list(recipes_sample.iloc[:, 1])[i]:^{max_lenght + 4}}|")  

|    id    |  minutes  |
|----------------------|
|  14003   |     35    |
|  268774  |    310    |
|  333262  |    120    |
|  217988  |     35    |
|  112486  |     80    |


2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [13]:
assert (
    show_info(
        name="george s at the cove black bean soup",
        steps=[
            "clean the leeks and discard the dark green portions",
            "cut the leeks lengthwise then into one-inch pieces",
            "melt the butter in a medium skillet , med",
        ],
        minutes=90,
        author_id=35193,
    )
    == '"George S At The Cove Black Bean Soup"\n\n1. Clean the leeks and discard the dark green portions\n2. Cut the leeks lengthwise then into one-inch pieces\n3. Melt the butter in a medium skillet , med\n----------\nАвтор: 35193\nСреднее время приготовления: 90 минут\n'
)

<IPython.core.display.Javascript object>

## Работа с регулярными выражениями

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [118]:
tree = ET.parse("steps_sample.xml")
root = tree.getroot()

recipe_id = 25082
steps = []
for recipe in root.findall('recipe'):
    if recipe.find('id').text == str(recipe_id):
        steps = [step.text.strip() for step in recipe.find('steps').findall('step')]

pattern = r'\d+\s+(?:hour|hours|minute|minutes)'
for step in steps:
    matches = re.findall(pattern, step)
    if matches:
        print(matches)

['20 minute']
['10 minute']
['2 hour']
['10 minute']
['20 minute', '30 minute']


4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [120]:
pattern = r'^this([\w\d\s]+),\s?but'
counter = 0
for desc in recipes['description']:
    if isinstance(desc, str):
        match = re.search(pattern, desc)
        if match and counter != 5:
            print(match.group())
            counter += 1
#если убрать счетчик counter из условия, выдаст все совпадения

this is a great meal eaten the same day ,but
this was adapted from a recipe i found on the net, but
this is kind of similar to some of the other versions out there, but
this is a moist, but
this pie does not have very many ingredients in it, but


5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [121]:
recipe_id = 72367
steps = []
for recipe in root.findall('recipe'):
    if recipe.find('id').text == str(recipe_id):
        steps = [step.text.strip() for step in recipe.find('steps').findall('step')]

pattern = r'(\d+)\s?/\s?(\d+)'
i = -1
for step in steps:
    i += 1
    match = re.search(pattern, step)
    if match:
        new_s = step[:match.start() + 1] + match.group(1) + '/' + match.group(2) + step[match.end():]
        steps[i] = new_s

for step in steps:
    print(step)

mix butter , flour , 11/3 c
sugar and 1-11/4 t
vanilla
press into greased 9" springform pan
mix cream cheese , 11/4 c
sugar , eggs and 11/2 t
vanilla beating until fluffy
pour over dough
combine apples , 11/3 c
sugar and cinnamon
arrange on top of cream cheese mixture and sprinkle with almonds
bake at 350 for 45-55 minutes , or until tester comes out clean


### Сегментация текста

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [125]:
def tokenize_to_words(s):
    words = word_tokenize(s.lower())
    words = [word for word in words if word.isalpha()]
    return words


tree = ET.parse("steps_sample.xml")
root = tree.getroot()

all_words = []
for recipe in root.findall('recipe'):
    steps = [step.text.strip() for step in recipe.find('steps').findall('step')]
    for step in steps:
        all_words.extend(tokenize_to_words(step))

unique_words = set(all_words)

print("Количество уникальных слов:", len(unique_words))

Количество уникальных слов: 14926


7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

8\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.


In [127]:
def pos_tag_sentence(sentence):
    tokens = word_tokenize(sentence)
    pos_tags = pos_tag(tokens)
    return pos_tags

try:
    pos_tag_sentence('')
except LookupError:
    import nltk
    nltk.download('averaged_perceptron_tagger')

def process(pos_tags):
    output = ""
    for token, pos_tag in pos_tags:
        output += f"{pos_tag:<5}"
    output += "\n"
    
    for token, _ in pos_tags:
        output += f"{token} "
    
    return output


recipe_id = 241106
recipes = pd.read_csv('recipes_sample.csv', header=0)
recipe_data = recipes[recipes['id'] == recipe_id]

print(process(pos_tag_sentence(recipe_data['name'].values[0])))

JJ   NNS  IN   NNS  VBP  JJ   CC   JJ   NNS  
eggplant steaks with chickpeas feta cheese and black olives 
